# Continuous Integration/Continuous Deployment (CI/CD)

## Continuous Integration (CI)
Change management is essential in software development. Once you make a change, you need to merge your changes where other people in your team can see it and ultimately use it. One of the most prominent version control applications in Github.

With Continuous Integration, each time you integrate your code with the source, an automatic *build* process should be triggered that compiles the code with new changes and possibly runs tests.

The main goal in this step is to catch and address the integration errors as early as possible in the development lifecycle, reducing the risk of conflicts and ensuring that the codebase remains stable.

## Continuous Deployment (CD)

After the build, we need to make the executables available for users in a target environment as well as adjust server configurations, transfer any files and update the databases if applicable. 


### Example - AWS CodeBuild

Here, we are going to dicuss a common set up for a CI/CD pipeline that uses [AWS CodeBuild](https://docs.aws.amazon.com/codebuild/latest/userguide/welcome.html).

In this case, code pushed to a version control system automatically triggers a CI/CD pipeline that builds the code on relevant cloud accounts. The repo includes <small>`buildspec.yaml`</small> and <small>`deployspec.yaml`</small> files that define how our application should be built and deployed.
They define the build and deployments environment, the steps required to compile and deploy our code, and any dependencies and configurations.

Here is an example of how a <small>`buildspec.yaml`</small> file looks like: ([link to AWS cumentaiton](https://docs.aws.amazon.com/codebuild/latest/userguide/getting-started-cli-create-build-spec.html))
``` yaml
version: 0.2 # the version of the build spec standard being use

phases: # the build phases during which you can instruct CodeBuild to run commands 
  install:
    runtime-versions:
      java: corretto11
  pre_build:
    commands:
      - echo Nothing to do in the pre_build phase... # the output of these echo commands can help you better understand how CodeBuild runs commands and in which orde
  build:
    commands:
      - echo Build started on `date`
      - mvn install # instructs Apache Maven to compile, test, and package the compiled Java class files into a build output artifact
  post_build:
    commands:
      - echo Build completed on `date`
artifacts: # the set of build output artifacts that CodeBuild uploads to the output bucket.
  files:
    - target/messageUtil-1.0.jar
```

What Happens During <small>`mvn install`</small> command?
- Apache Maven is a build automation tool primarily used for Java projects. It helps manage project builds, dependencies, and documentation.
- <small>`mvn`</small>: This is the command-line tool used to invoke Maven.
- During this command: 
  - Maven compiles the source code of the project
  - Maven runs unit tests on the compiled code.
  - Maven packages the compiled code and any other necessary resources into a build output artifact.
  - After packaging, Maven installs the artifact into the local Maven repository on your machine.


<style>
/* CSS to change font size of code blocks */
pre {
    font-size: 12px; /* Adjust the font size as needed */
}
</style>

## Containerization

A <span style='color: #32cd32;'>**container**</span> is a lightweight, standalone, and executable software package that includes everything needed to run a piece of software: the code, runtime, system tools, libraries, and settings. <br>
It provides isolated environments to run software services independently and optimize use of resources from *one piece of hardware*.

<span style='color: #32cd32;'>**Docker**</span> is a platform that allows you to automate the deployment of applications inside containers.

A <span style='color: #32cd32;'>**Dockerfile**</span> is a text file that contains a set of instructions for building a Docker image. It specifies what base image to use, what dependencies to install, and how to configure the application. The Dockerfile itself is not executable; it just defines the steps needed to create an image.

A <span style='color: #32cd32;'>**Docker Image**</span> is the result of executing the instructions in a Dockerfile. It is a static, read-only template that contains everything needed to run an application (code, runtime, libraries, etc.). You create Docker images using the <small>`docker build`</small> command, which reads the Dockerfile and produces an image.

Once you have a Docker image, you create <span style='color: #32cd32;'>**Docker Containers**</span> from it. A container is a running instance of the image, with its own isolated environment.

An <span style='color: #32cd32;'>**Docker Image Tag**</span> is a label attached to a Docker image that that is used to differentiate between various versions or variants of an image. For example, an image might be tagged as latest, v1.0, or stable, and each tag points to a different version of the image. Tags are useful for versioning and deploying specific versions of an image.


Here's a simple example of a Dockerfile for a Python application:
``` dockerfile
# Use an official Python runtime as a parent image
# The base image is often a standard image such as an operating system image (e.g., ubuntu, alpine) or a specialized image (e.g., node, python)
FROM python:3.9-slim

# Set the working directory inside the Docker image. Commands that follow this instruction will be executed in this directory.
WORKDIR /app

# Copy the current directory contents from the host machine into the Docker image? at /app
COPY . /app

# Install any needed packages specified in requirements.txt
# The `RUN` instruction is used to execute commands in the image, such as installing software or dependencies.
RUN pip install --no-cache-dir -r requirements.txt

# Make port 5000 available to the world outside this container
# The `EXPOSE` instruction indicates which ports the container will listen on at runtime.
EXPOSE 5000

# Define environment variable
# The `ENV` instruction sets environment variables inside the Docker image.
ENV FLASK_ENV=production

# Run app.py when the container launches
CMD ["python", "app.py"]
```

To generate a Docker image from the provided Dockerfile:
- Make sure you have the Dockerfile saved in your project directory.
- Ensure you have a <small>`requirements.txt`</small> file in the same directory.
- Ensure you have the <small>`app.py`</small> file or any other files that your application needs.
- Open a terminal or command prompt and navigate to the directory where your Dockerfile is located
- Run <small>`docker build -t my-python-app .`</small> to to build the image

A Docker image is a binary object that contains all the components needed to run a software application, including the code, runtime, libraries, and dependencies. It is not a script or text file that you can open and edit directly. <br>
Docker images consist of a series of layers. Each layer represents a set of changes or additions to the image. The image layers are managed by Docker and stored in a registry. Docker Images are identified and managed by Docker using tags and IDs rather than file extensions. <br>
Docker images stored on your local machine or in a Docker registry.You interact with Docker images using Docker commands rather than directly manipulating image files.

Here’s a conceptual view of what the image might include:
``` sql
+------------------------------------+
| Python 3.9-Slim Base Image          |
|                                     | 
| /app                                |
| ├── app.py                          |
| ├── requirements.txt                |
| └── [Other files from current dir]  |
|                                     |
| Installed Packages:                 |
| └── (Packages from requirements.txt)|
|                                     |
| Exposed Ports:                      |
| └── 5000                            |
|                                     |
| Environment Variables:              |
| └── FLASK_ENV=production            |
|                                     |
| Default Command:                    |
| └── python app.py                   |
+------------------------------------+
```

Start and run a container with <small>`docker run -p 5000:5000 my-python-app`</small><br>
The application running inside the container will be accessible on port 5000 of your host machine.

<style>
/* CSS to change font size of code blocks */
pre {
    font-size: 12px; /* Adjust the font size as needed */
}
</style>


#### What is Amazon Elastic Container Registry (ECR)?

ECR is a fully managed container registry service provided by AWS. It allows you to store, manage, and deploy Docker container images.


### Implementation

An example of a <small>`buildspec.yaml`</small> configuring AWS CodeBuild to build and push a Docker image to a container registry like ECR.

``` yaml
version: 0.2

phases:
  pre_build:
    commands:
      - echo Logging in to Amazon ECR...
      - $(aws ecr get-login --no-include-email --region $AWS_DEFAULT_REGION) 
      - echo Setting environment variables
      - REPOSITORY_URI=your-account-id.dkr.ecr.your-region.amazonaws.com/your-repository-name # reference the location of your Docker images in ECR
      - IMAGE_TAG=${CODEBUILD_RESOLVED_SOURCE_VERSION} # sets the `IMAGE_TAG` variable to the source version of the code
      - echo $IMAGE_TAG # This tag is usually a commit hash or build number.

  build:
    commands:
      - echo Building the Docker image...
      - docker build -t $REPOSITORY_URI:$IMAGE_TAG . # builds the Docker image from the Dockerfile in the current directory (`.`) and tags it with `REPOSITORY_URI:$IMAGE_TAG`
      - docker tag $REPOSITORY_URI:$IMAGE_TAG $REPOSITORY_URI:latest # tags the newly built Docker image with an additional latest tag. This helps to identify the most recent version of the image.

  post_build:
    commands:
      - echo Pushing the Docker images to Amazon ECR...
      - docker push $REPOSITORY_URI:$IMAGE_TAG # Pushes the Docker image with the specific tag ($IMAGE_TAG) to ECR
      - docker push $REPOSITORY_URI:latest # Pushes the Docker image with the latest tag to ECR

artifacts: # the files or outputs that result from a build process and are meant to be saved, shared, or used in subsequent steps of a pipeline
  files: []

```

- The URI of your container repository in ECR typically looks like this: <small>`aws_account_id.dkr.ecr.region.amazonaws.com/repository_name`</small>
  - <small>`aws_account_id`</small>: Your AWS account ID.
  - <small>`region`</small>: The AWS region where your ECR repository is located.
  - <small>`repository_name`</small>: The name of your ECR repository.

- **Tag Overwriting**: The <small>`latest`</small> tag is a floating tag. Each time you build and push an image with the <small>`latest`</small> tag, it updates the <small>`latest`</small> tag to point to the most recently pushed image. The previous image that was tagged as <small>`latest`</small> will still exist in the repository, but it will no longer be associated with the <small>`latest`</small> tag.

<style>
/* CSS to change font size of code blocks */
pre {
    font-size: 12px; /* Adjust the font size as needed */
}
</style>